In [17]:
# Importación de librerías generales
import pandas as pd
pd.options.mode.chained_assignment = None  # Avoid warning messages
import numpy as np

import shap
import transformers

import spacy

# Lectura de los dataset
data = pd.read_csv('tweets_prepro.csv', lineterminator='\n')

# LEMATIZACIÓN Y REDUCCIÓN

In [18]:
data

,tweet,date,author,influence,score,polarity,dim,atrib,stakeholder,tag,theme,tweet_prepro,tweet_prepro_english
0,Saludamos el acuerdo logrado entre TGP y Comun...,2017-12-07,HumbertoTYL,6,1372.0,2.0,DESCONOCIDO,Desconocido,Desconocido,Acuerdo Desarrollo Chinquintirca,bcp,saludamos el acuerdo logrado entre tgp y comun...,We welcome the agreement reached between tgp a...
1,"RT @meiermq: \""First Capital intervino de ?ase...",2017-12-17,Jaicec,6,759.0,-1.0,DESCONOCIDO,Desconocido,Desconocido,Asesoría Financiera PPK,bcp,"[REF] ""first capital intervino de ?asesor fina...","[REF] ""First capital intervened from?exclusive..."
2,#Ayacucho : Comunidades afectadas por fugas de...,2017-11-09,NoticiasSER,7,737.0,-2.0,DESCONOCIDO,Desconocido,Desconocido,Fugas de Gas,bcp,#ayacucho : comunidades afectadas por fugas de...,#ayacucho : communities affected by gas leaks ...
3,"RT @HumbertoTYL: Comuneros de Chiquintirca, An...",2017-10-12,BancadaFAperu,6,651.0,-2.0,DESCONOCIDO,Desconocido,Desconocido,Fugas de Gas,bcp,"[REF]comuneros de chiquintirca, anco - ayacuch...","[REF]communeros of chiquintirca, anco - ayacuc..."
4,"RT @IsraelLazoJ: En #ComisiónVraem, el Gerente...",2017-12-13,jalmenaram,6,610.0,-2.0,DESCONOCIDO,Desconocido,Desconocido,Fugas de Gas,bcp,"[REF]en #comisiónvraem, el gerente general de ...","[REF]in #comisiónvraem, the general manager of..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83131,Nuevos Soles en la Guía Repsol 2015: Se acaba ...,2014-12-02,Gastronomiaycia,10,271566.0,2.0,guia repsol,Desconocido,AMBITO EMPRESARIAL,Soles Repsol,rep,nuevos soles en la guía [SUB] 2015: se acaba d...,new suns in the guide [SUB] 2015: the guide [S...
83132,EL Tribunal Superior de Canarias permite a Rep...,2014-12-09,ElHuffPost,10,271434.0,1.0,participación comunidad,generación de riqueza e ingresos,MMCC Y LIDERES DE OPINION,Prospecciones Canarias,rep,el tribunal superior de canarias permite a [SU...,the canary superior court allows [SUB] to cont...
83133,RT @box_repsol: En el descanso de @El_Hormigue...,2014-12-03,box_repsol,5,258629.0,1.0,box repsol,Desconocido,EMPRESA,Box Repsol,rep,rt [SUB] en el descanso de [REF] enseñandole l...,rt [SUB] in the rest of [REF] teaching him the...
83134,Repsol comprará a la canadiense Talisman por U...,2014-12-16,Gestionpe,10,256726.0,1.0,participación comunidad,generación de riqueza e ingresos,MMCC Y LIDERES DE OPINION,Adquisición Talisman Energy,rep,[SUB] comprará a la canadiense talisman por us...,[SUB] will buy the Canadian talisman for US$ 1...


In [19]:
# Se eliminan las palabras que empiezan por # o @
data.tweet_prepro = data.tweet_prepro.str.replace(r'(#)[^ ]*', '', regex = True)
data.tweet_prepro = data.tweet_prepro.str.replace(r'(@)[^ ]*', '', regex = True)

# Se eliminan las máscaras
sust =[
('[REF]',''),
('[LIN]',''),
('[SUB]',''),
('[MED]',''),
('[[]]',''),
('[?]',''),
(' : ',' '),
('  ',' '),]

for couple in sust:
    data['tweet_prepro'] = data['tweet_prepro'].str.replace(couple[0], couple[1], regex=True)

Possible nested set at position 1


In [22]:
nlp = spacy.load('es_core_news_lg')
lemmatized_sentences = []

for sentence in data['tweet_prepro']: 
    text = sentence
    sentence = nlp(text)

    # Las anteriores se lematizan
    lemmas = [word.lemma_.lower() for word in sentence]


    prep = ' '.join(lemmas)

    # Se ajustan los espacios
    sust =[
    (' ,',','),
    (' .','.'),
    (' ;',';'),
    (' ?','?'),
    (' !','!'),
    (' :',':'),
    (' ( ', ' ('),
    (' ) ', ') '),
    (' "', '"'), 
    ('" ', '"'),
    ('  ', ' ')
    ]

    for couple in sust:
        prep = prep.replace(couple[0], couple[1])
        
    lemmatized_sentences.append(prep)
    
data['tweet_prepro'] = lemmatized_sentences

# Se eliminan dobles espacios
data.tweet_prepro = data.tweet_prepro.str.replace(' +', ' ', regex = True)

# Se eliminan filas donde el preproceso ha dejado mensajes vacíos
data = data[data['tweet_prepro'].notnull()]

In [23]:
data.to_csv('lematizado-generico.csv', line_terminator='\n', index=False)

In [25]:
data.tweet_prepro[0]

'saludar el acuerdo logrado entre tgp y comunidad de chiquintirca. no obstante, aún faltar más de 15 punto por resolver. él haber acompañar hasta hoy y él seguir hacer.'